## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [56]:
from dotenv import load_dotenv
import os   
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')


In [57]:
from langchain_groq import ChatGroq

model = ChatGroq(model='llama-3.1-8b-instant', groq_api_key=groq_api_key)

model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001EEFB6912D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EEFB6936D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [58]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hii, my name is chandan and i'm a software engineer.")])

AIMessage(content="Hello Chandan. It's nice to meet you. As a software engineer, you must be involved in developing and maintaining software systems. What kind of projects or technologies have you been working on lately?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 50, 'total_tokens': 91, 'completion_time': 0.060663926, 'completion_tokens_details': None, 'prompt_time': 0.002349333, 'prompt_tokens_details': None, 'queue_time': 0.055347527, 'total_time': 0.063013259}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b9658-8d9e-75b2-9322-eccfb0512f55-0', usage_metadata={'input_tokens': 50, 'output_tokens': 41, 'total_tokens': 91})

In [59]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hii, my name is chandan and i'm a software engineer."),
    AIMessage(content="Nice to meet you, Chandan. Welcome to our conversation. What brings you here today? Are you working on a new project, or do you have some questions about software engineering? I'm all ears and ready to help."),
    HumanMessage(content='Hey, what is my name and what do i do ??')
])

AIMessage(content="Your name is Chandan, and you're a software engineer. How's life as a software engineer treating you, Chandan?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 118, 'total_tokens': 145, 'completion_time': 0.03458488, 'completion_tokens_details': None, 'prompt_time': 0.107352282, 'prompt_tokens_details': None, 'queue_time': 0.051888858, 'total_time': 0.141937162}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b9658-902b-73c3-80ac-61c6cb7c5f2b-0', usage_metadata={'input_tokens': 118, 'output_tokens': 27, 'total_tokens': 145})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [60]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [61]:
## set configuration
config = {'configurable': {'session_id':'chat1'}}

In [62]:
response = with_message_history.invoke([
    HumanMessage(content="Hii, my name is chandan and i'm a software engineer.")
],config=config)

In [63]:
response

AIMessage(content="Nice to meet you, Chandan! How's your day going so far as a software engineer? Do you have any exciting projects or challenges you're working on?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 50, 'total_tokens': 84, 'completion_time': 0.053173144, 'completion_tokens_details': None, 'prompt_time': 0.002742223, 'prompt_tokens_details': None, 'queue_time': 0.067785286, 'total_time': 0.055915367}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b9658-918e-7b41-b879-5ffd92012b30-0', usage_metadata={'input_tokens': 50, 'output_tokens': 34, 'total_tokens': 84})

In [64]:
response.content

"Nice to meet you, Chandan! How's your day going so far as a software engineer? Do you have any exciting projects or challenges you're working on?"

In [65]:
with_message_history.invoke(
    [HumanMessage(content='What is my name ??')],
    config=config
)

AIMessage(content="Your name is Chandan, and you're a software engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 98, 'total_tokens': 112, 'completion_time': 0.019650937, 'completion_tokens_details': None, 'prompt_time': 0.005392812, 'prompt_tokens_details': None, 'queue_time': 0.056258518, 'total_time': 0.025043749}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b9658-92ca-7171-a142-94f025c26902-0', usage_metadata={'input_tokens': 98, 'output_tokens': 14, 'total_tokens': 112})

In [66]:
## change the config --> session id

config1 = {'configurable': {'session_id':'chat2'}}

response = with_message_history.invoke(
    [HumanMessage(content='what is my name ??')],
    config=config1
)
response

AIMessage(content="I don't have any information about your name. I'm a conversational AI, and our conversation just started. If you'd like to tell me your name, I'd be happy to learn it.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 40, 'total_tokens': 82, 'completion_time': 0.066784692, 'completion_tokens_details': None, 'prompt_time': 0.001833045, 'prompt_tokens_details': None, 'queue_time': 0.055223915, 'total_time': 0.068617737}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b9658-9386-7b10-ae02-ee82e2a8b9d1-0', usage_metadata={'input_tokens': 40, 'output_tokens': 42, 'total_tokens': 82})

In [67]:
response.content

"I don't have any information about your name. I'm a conversational AI, and our conversation just started. If you'd like to tell me your name, I'd be happy to learn it."

In [68]:
response = with_message_history.invoke(
    [HumanMessage(content='My name is Shyam..')],
    config=config1
)

response.content

"Nice to meet you, Shyam. I'll remember that for our conversation. How are you doing today?"

In [69]:
response = with_message_history.invoke(
    [HumanMessage(content='what is my name ??')],
    config=config1
)

response.content

'I remember, your name is Shyam.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [70]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [('system', 'You are helpful assistant. Answer all the questions on our best possibility..'),
     MessagesPlaceholder(variable_name='message')]
)

chain = prompt | model

In [71]:
chain.invoke({'message': [HumanMessage(content='Hii, my name is chandan.')]})

AIMessage(content='Nice to meet you, Chandan! How are you doing today? Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 58, 'total_tokens': 88, 'completion_time': 0.050582825, 'completion_tokens_details': None, 'prompt_time': 0.00388542, 'prompt_tokens_details': None, 'queue_time': 0.05638558, 'total_time': 0.054468245}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b9658-96a5-7923-bf7a-632553ae53f9-0', usage_metadata={'input_tokens': 58, 'output_tokens': 30, 'total_tokens': 88})

In [72]:
chain.invoke({'message': [HumanMessage(content='You generate some kind of multiple emoji..')]})

AIMessage(content='🌟💻📚💡🎉👍', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 57, 'total_tokens': 74, 'completion_time': 0.089859359, 'completion_tokens_details': None, 'prompt_time': 0.003845798, 'prompt_tokens_details': None, 'queue_time': 0.061132501, 'total_time': 0.093705157}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b9658-9791-7633-9808-bcb1ee904513-0', usage_metadata={'input_tokens': 57, 'output_tokens': 17, 'total_tokens': 74})

In [73]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history=get_session_history)

In [74]:
config = {'configurable': {'session_id':'chat3'}}

response = with_message_history.invoke(
    [HumanMessage(content='Hii, my name is chandan..')],
    config=config
)

response

AIMessage(content="Hello Chandan, it's nice to meet you. I'm here to help with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 58, 'total_tokens': 94, 'completion_time': 0.130427547, 'completion_tokens_details': None, 'prompt_time': 0.004838868, 'prompt_tokens_details': None, 'queue_time': 0.057127912, 'total_time': 0.135266415}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b9658-9897-7720-9f16-daeee1598880-0', usage_metadata={'input_tokens': 58, 'output_tokens': 36, 'total_tokens': 94})

In [75]:
## add more complexity

prompt = ChatPromptTemplate.from_messages(
    [('system', 'You are helpful assistant and answer all the quesions on your best possiblity in {language} language.'),
    MessagesPlaceholder(variable_name='messages')]
)

chain = prompt | model

In [76]:
response = chain.invoke({'messages':[HumanMessage(content='Hii, my name is chandan.')],
                         'language': 'Hindi'})

response.content

'नमस्ते चंदन जी, मैं आपका सहायक हूँ। आपको किस बारे में जानकारी चाहिए?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [77]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
)

In [78]:
config = {'configurable': {'session_id':'chat4'}}

response = with_message_history.invoke(
    {'messages':[HumanMessage(content='Hii, i am chandan.')], 'language':'Hindi'},
    config=config
)
response.content

'नमस्ते चंदन, मैं आपकी सहायता करने के लिए यहाँ हूँ। क्या आपके पास कोई प्रश्न है, या आप कुछ जानना चाहते हैं?'

In [79]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content='What is my name??')], 'language': 'Hindi'},
    config=config
)

response.content

'तुम्हारा नाम चंदन है।'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

**'trim_messages'** helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [80]:
## In that token size is 70 so do not trim any messages ...

from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [81]:
## In that we decrease token size is 70 to 35, so it trim messages ...

from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=35,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [82]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter('messages')| trimmer) | prompt | model
)

response = chain.invoke({
    'messages': messages + [HumanMessage(content='What type of ice cream i like???')], 'language':'English'
})

response.content

"I'm happy to help, but I'm a large language model, I don't know your personal preferences yet. Would you like to share what type of ice cream you like, or would you like me to suggest some popular flavors?"

In [83]:
## for below context increase token size

from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [84]:

chain = (
    RunnablePassthrough.assign(messages=itemgetter('messages')| trimmer) | prompt | model
)

response = chain.invoke({
    'messages': messages + [HumanMessage(content='What type of ice cream i like???')], 'language':'English'
})

response.content

'You like vanilla ice cream!'

In [85]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [86]:

response = with_message_history.invoke({
    'messages': messages + [HumanMessage(content='What type of ice cream i like???')], 'language':'English'
}, config=config)

response.content

'You like vanilla ice cream!'

In [87]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked the math problem: 2 + 2.'